In [5]:
import pandas as pd
data = pd.read_csv('Sample_1.csv')


In [6]:
data.columns

Index(['p_id', 'age', 'female', 'ethnicity', 'grip', 'chest', 'e_anx_1',
       'e_anx_2', 'e_anx_3', 'e_anx_4', 'e_anx_5', 'e_anx_6', 'e_anx_7',
       'e_anx_8', 'e_anx_9', 'e_anx_10', 'e_dep_1', 'e_dep_2', 'e_dep_3',
       'e_dep_4', 'e_dep_5', 'e_dep_6', 'e_dep_7', 'e_dep_8', 'e_dep_9',
       'e_dep_10', 'e_fear_1', 'e_fear_2', 'e_fear_3', 'e_fear_4', 'e_fear_5',
       'e_fear_6', 'e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10', 'e_sen_1',
       'e_sen_2', 'e_sen_3', 'e_sen_4', 'e_sen_5', 'e_sen_6', 'e_sen_7',
       'e_sen_8', 'e_sen_9', 'e_sen_10'],
      dtype='object')

In [7]:
mean_anx = data[['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10']].mean(axis=1)
mean_dep = data[['e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10']].mean(axis=1)
mean_fear = data[['e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10']].mean(axis=1)
mean_sen = data[['e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']].mean(axis=1)

In [8]:
data['mean_anx']=mean_anx
data['mean_dep']=mean_dep
data['mean_fear']=mean_fear
data['mean_sen']=mean_sen

In [9]:
to_drop = ['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10',\
          'e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10',\
          'e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10',\
          'e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']
data.drop(columns=to_drop, inplace=True)

In [10]:
data.columns

Index(['p_id', 'age', 'female', 'ethnicity', 'grip', 'chest', 'mean_anx',
       'mean_dep', 'mean_fear', 'mean_sen'],
      dtype='object')

In [89]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [90]:
# Drop the rows with missing values
data = data.dropna()

# Split the dataset into features and target variable
X = data.drop(['mean_fear'], axis=1) # Drop fearfulness score as it is the target variable
y = data['mean_fear']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#print(r)

In [91]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [92]:
# Train the SVM model
svm_reg = SVR(kernel='linear', C=1)
svm_reg.fit(X_train, y_train)

SVR(C=1, kernel='linear')

In [93]:
# Predict the fearfulness score of the test set
y_pred = svm_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Evaluate the model performance
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R^2:", r2)

Mean Squared Error: 0.3009528199370184
Mean Absolute Error: 0.4395422440575617
R^2: 0.05677939819155853


In [95]:
#SVM classification

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Convert fearfulness scores into three categories
threshold1 = data['mean_fear'].quantile(0.33)
threshold2 = data['mean_fear'].quantile(0.66)
data['fearfulness_category'] = pd.cut(data['mean_fear'], bins=[-float('inf'), threshold1, threshold2, float('inf')], labels=['low', 'medium', 'high'])

# Drop the rows with missing values
data = data.dropna()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['mean_fear', 'fearfulness_category'], axis=1), data['fearfulness_category'], test_size=0.2, random_state=42)

# Train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = svm.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')

Accuracy: 0.3142857142857143
F1-score: 0.30727513227513226


In [1]:
import pyspark
print(pyspark.__version__)


3.3.2


In [11]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.sql.functions import percentile_approx

In [14]:
# Convert fearfulness scores into three categories
threshold1 = data.selectExpr('percentile_approx(mean_fear, 0.33)').collect()[0][0]
threshold2 = data.selectExpr('percentile_approx(mean_fear, 0.66)').collect()[0][0]
#data = df_with_means.withColumn('fearfulness_category', 
#                       when(col('mean_fear') <= threshold1, 'low')
#                       .when((col('mean_fear') > threshold1) & (col('mean_fear') <= threshold2), 'medium')
#                       .otherwise('high'))



AttributeError: 'DataFrame' object has no attribute 'selectExpr'

In [ ]:
#threshold1 = data.approxQuantile('mean_fear', [0.33], 0)[0]
#threshold2 = data.approxQuantile('mean_fear', [0.66], 0)[0]
#data = data.withColumn('fearfulness_category',
#when(col('mean_fear') <= threshold1, 'low')
#.when((col('mean_fear') > threshold1) & (col('mean_fear') <= threshold2), 'medium')
#.otherwise('high'))

data = data.na.drop()

X_train, X_test = data.drop('mean_fear', 'fearfulness_category').randomSplit([0.8, 0.2], seed=42)
y_train = data.select('fearfulness_category')

indexer = StringIndexer(inputCol='fearfulness_category', outputCol='label')
indexed_train_data = indexer.fit(y_train).transform(y_train)

svm = LinearSVC(maxIter=10, regParam=0.1)
svm_model = svm.fit(X_train.join(indexed_train_data, X_train.index == indexed_train_data.index, 'inner').drop('index'))

y_pred = svm_model.transform(X_test)

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
accuracy = evaluator.evaluate(y_pred)
f1_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1 = f1_evaluator.evaluate(y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')